5주차 예습과제에서는 6주차 내용인 GPT의 생성 능력을 맛보고자 합니다. 사용할 모델은 koalpaca라는 모델인데, 한국어 버전의 alpaca 모델입니다.

alpaca는 llama라는 모델을 instruction fine-tuning 해서 사람의 지시를 잘 따르는 모델입니다.

llama라는 모델은 LLaMA-13B의 경우 GPT-3보다 10배이상 작음에도 불구하고 대부분의 평가서 GPT-3보다 우수한 성능을 보이며, 더 나아가 LLaMA-65B의 경우 대부분의 벤치마크에서 Chinchilla, Gopher, GPT-3, PaLM와 유사하거나 더 뛰어난 결과를 보이는 훌륭한 모델입니다.

그러면 지금부터 koalpaca로 생성 모델을 한번 경험해보도록 하겠습니다!

In [1]:
# GPU 켜져 있는지 확인

!nvidia-smi

Wed Feb 14 07:33:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are instal

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

peft는 parameter efficient fine-tuning의 약자로, 큰 모델을 코랩에서 돌릴 수 있도록 쪼개서 만든 것입니다.

오래 걸립니다!

In [4]:
from peft import PeftModel, PeftConfig

In [5]:
peft_model_id = "beomi/qlora-koalpaca-polyglot-12.8b-50step"
config = PeftConfig.from_pretrained(peft_model_id)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, quantization_config=bnb_config, device_map={"":0})
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/428 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/52.5k [00:00<?, ?B/s]

model-00001-of-00028.safetensors:   0%|          | 0.00/946M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 5120)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-39): 40 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=5120, out_features=15360, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                

next token generation

In [6]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 문제들을 해결할 수 있다"고 말했다.<|endoftext|>


 QA

In [7]:
def gen(x):
    q = f"### 질문: {x}\n\n### 답변:"
    # print(q)
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        max_new_tokens=50,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

In [8]:
gen('건강하게 살기 위한 세 가지 방법은?')

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:447: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 건강하게 살기 위한 세 가지 방법은?

### 답변: 건강하게 살기 위한 방법, 이라는 질문이 있었는데요, 사실 건강하게 살기 위해서 필요한 것은 세 가지가 아니라 열 가지도 넘습니다. 그런데 그렇게 얘기를 하면 사람들이 그걸 다 못 지키


top_k와 같은 parameter를 조절하면서 실험해보세요!

Q) top_k 는 무슨 parameter이며 어떻게 달라질까?

A) top_k는 GPT 모델에서 문장을 생성할 때 다음 단어를 선택할 때 고려하는 후보의 개수를 결정하는 하이퍼파라미터입니다. 이는 다음 단어를 예측할 때 확률 분포에서 상위 K개의 가능한 후보만을 고려하여 선택하는 것을 의미합니다.

보통 top_k 값이 클수록 모델이 더 많은 후보를 고려하므로 생성된 문장이 더 다양하고 예측이 더 무작위적이 될 수 있습니다. 하지만 너무 큰 top_k 값은 모델이 무작위성을 많이 가지게 되어 문장의 일관성을 잃을 수 있습니다.

그래서 top_k를 default인 50부터 5, 10, 20, 50, 100, 200 등 다양한 값을 실현해볼 예정

In [15]:
import time
import numpy as np

In [17]:
# top_k = 50

prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
start_time = time.time()

with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64, top_k=50)
  generated = tokenizer.batch_decode(gen_tokens)[0]

end_time = time.time()

print(generated)
print("실행시간 :", np.round(end_time - start_time, 7), " sec")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 문제의 해답을 얻을 수 있을 것으로 전망된다.IBM은 '엑소브레인'에 대해 "IBM은 '엑소브레인'을 개발하기 위해 방대한 데이터를 분석
실행시간 : 7.0751443  sec


In [18]:
# top_k = 5

start_time = time.time()

with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64, top_k=5)
  generated = tokenizer.batch_decode(gen_tokens)[0]

end_time = time.time()

print(generated)
print("실행시간 :", np.round(end_time - start_time, 7), " sec")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 문제들을 풀어낼 수 있을 것"이라고 말했다.<|endoftext|>
실행시간 : 4.1291599  sec


In [19]:
# top_k = 10

start_time = time.time()

with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64, top_k=10)
  generated = tokenizer.batch_decode(gen_tokens)[0]

end_time = time.time()

print(generated)
print("실행시간 :", np.round(end_time - start_time, 7), " sec")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 난제를 풀고, 더 나은 세상을 만들어 나가길 기대해봅니다.[김민수 기자 minsu@mbn.co.kr]<|endoftext|>
실행시간 : 6.9330263  sec


In [20]:
# top_k = 20

start_time = time.time()

with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64, top_k=20)
  generated = tokenizer.batch_decode(gen_tokens)[0]

end_time = time.time()

print(generated)
print("실행시간 :", np.round(end_time - start_time, 7), " sec")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 많은 난제를 해결할 것으로 기대되고 있다. 이에 따라 로봇이 산업 현장만이 아닌 일상 생활에도 본격적으로 진출할 날이 머지않았다는 전망이 나온다.​'
실행시간 : 6.7234039  sec


In [21]:
# top_k = 30

start_time = time.time()

with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64, top_k=30)
  generated = tokenizer.batch_decode(gen_tokens)[0]

end_time = time.time()

print(generated)
print("실행시간 :", np.round(end_time - start_time, 7), " sec")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 숙제들을 해결해 나가고 있습니다. ​​​​​​​​​​​​​​<|endoftext|>
실행시간 : 6.3265188  sec


In [22]:
# top_k = 100

start_time = time.time()

with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64, top_k=100)
  generated = tokenizer.batch_decode(gen_tokens)[0]

end_time = time.time()

print(generated)
print("실행시간 :", np.round(end_time - start_time, 7), " sec")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 난제들을 해결하는 '슈퍼 휴먼'이 탄생할 것이다. ​​​AI와 인간은 공생해야 한다. ​AI와 공존하는 인간과 AI의 공존
실행시간 : 8.7102127  sec


In [23]:
# top_k = 200

start_time = time.time()

with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64, top_k=200)
  generated = tokenizer.batch_decode(gen_tokens)[0]

end_time = time.time()

print(generated)
print("실행시간 :", np.round(end_time - start_time, 7), " sec")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 문제들을 해결할 것"이라고 전했다.​또한 그는 "인텔은 현재 인공지능 개발 분야에서 세계를 선도하고 있다"며 "이 인공지능의 미래를 인텔의 설계 기술
실행시간 : 7.5706663  sec


In [25]:
# top_k = 200, max_length longer

start_time = time.time()

with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=128, top_k=200)
  generated = tokenizer.batch_decode(gen_tokens)[0]

end_time = time.time()

print(generated)
print("실행시간 :", np.round(end_time - start_time, 7), " sec")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 난제를 해결할 수 있다.'고 말했습니다. '인공지능의 발전은 인간이 만들어낸 결과물이다'는 사실을 다시금 확인했습니다. 인공지능이 발전되기 위해서는 인간의 사고능력을 뛰어넘는 지능을 보유할 수 있는 머신러닝 관련 최신기술들이 뒷받침되어야 하기 때문입니다. ​현재 우리가 인공지능하면 떠오르는 대표주자는 구글의 알파고와 IBM의 왓슨이 있습니다. 알파고는 한 번이라도 게임을 해본 사람의 게임방법
실행시간 : 14.5947478  sec


# Summary

k가 증가할 수록 문장이 길어진다. k=10,20 정도에서 깔끔한 문장이 생성되고, 그 이상은 필요 이상의 말을 하는 듯 한 느낌의 문장들을 생성해내거나, 아예 원하는 주제에서 벗어나기도 한다

max_length=64로 통일하고 실험했을 때, k가 30 이상에서 원래의 생성하려던 문장보다 max_length에 의한 제한에 의해 문장이 중간에 잘리는 경우가 다수였다.

또한, 시드 값의 영향을 받는지 알 수 없으나, 생성 시마다 결과가 달라진다.